<a href="https://colab.research.google.com/github/Mahdi-Golizadeh/INTENT-DETECTION/blob/main/intent_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install necessary libraries

In [1]:
!pip install -q transformers datasets evaluate sentencepiece accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 77.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

import required libraries

In [2]:
import transformers
import datasets
import evaluate
import numpy as np
import sentencepiece

download the dataset which we want to do classification on it

In [3]:
intent = datasets.load_dataset("AmazonScience/massive", "fa-IR")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2033 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2974 [00:00<?, ? examples/s]

to see dataset overall srtucture

In [4]:
intent

DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2974
    })
})

a sample of the dataset

In [5]:
intent["train"][0]

{'id': '1',
 'locale': 'fa-IR',
 'partition': 'train',
 'scenario': 16,
 'intent': 48,
 'utt': 'مرا جمعه ساعت نه صبح بیدار کن',
 'annot_utt': 'مرا [date : جمعه] ساعت [time : نه صبح] بیدار کن',
 'worker_id': '3',
 'slot_method': {'slot': ['time', 'date'],
  'method': ['translation', 'translation']},
 'judgments': {'worker_id': ['3', '8', '14'],
  'intent_score': [1, 1, 1],
  'slots_score': [1, 1, 1],
  'grammar_score': [4, 4, 4],
  'spelling_score': [2, 2, 2],
  'language_identification': ['target', 'target', 'target']}}

according to the sample we only need "utt" and "intent" keys

In [6]:
intent["train"][0]["utt"], intent["train"][0]["intent"]

('مرا جمعه ساعت نه صبح بیدار کن', 48)

to see the number of classes

In [7]:
len(intent["train"].features["intent"].names)

60

since we want to classify in persian language we need something like bert which is specialized for farsi language like the following checkpoint

In [8]:
checkpoint = "HooshvareLab/bert-fa-base-uncased-clf-persiannews"

load tokenizer for the specified checkpoint

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

to have faster training we truncated the length of sequence to 64 it can be changed to the maximum of the model length

In [10]:
max_token_length= 64
def tokenize_func(sample):
    return tokenizer(sample["utt"],max_length=max_token_length, truncation= True)

In [11]:
tokenized_intent = intent.map(tokenize_func, batched= True)

Map:   0%|          | 0/11514 [00:00<?, ? examples/s]

Map:   0%|          | 0/2033 [00:00<?, ? examples/s]

Map:   0%|          | 0/2974 [00:00<?, ? examples/s]

In [12]:
tokenized_intent = tokenized_intent.rename_column("intent", "labels")

In [13]:
tokenized_intent["train"][0]

{'id': '1',
 'locale': 'fa-IR',
 'partition': 'train',
 'scenario': 16,
 'labels': 48,
 'utt': 'مرا جمعه ساعت نه صبح بیدار کن',
 'annot_utt': 'مرا [date : جمعه] ساعت [time : نه صبح] بیدار کن',
 'worker_id': '3',
 'slot_method': {'slot': ['time', 'date'],
  'method': ['translation', 'translation']},
 'judgments': {'worker_id': ['3', '8', '14'],
  'intent_score': [1, 1, 1],
  'slots_score': [1, 1, 1],
  'grammar_score': [4, 4, 4],
  'spelling_score': [2, 2, 2],
  'language_identification': ['target', 'target', 'target']},
 'input_ids': [2, 3306, 5682, 3551, 3254, 5072, 9146, 2860, 4],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

to have better batching length we use datacollator

In [14]:
data_collator = transformers.DataCollatorWithPadding(
    tokenizer= tokenizer
)

use accuracy metrics for evaluation

In [15]:
accuracy = evaluate.load("accuracy")

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

define the labels and equivalent ids

In [17]:
id2label = {i: label for i, label in enumerate(intent["train"].features["intent"].names)}
label2id = {k: v for v, k in id2label.items()}

load model checkpoint for finetuning

In [18]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                                        num_labels=len(intent["train"].features["intent"].names),
                                                                        ignore_mismatched_sizes=True,
                                                                        id2label=id2label,
                                                                        label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased-clf-persiannews and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([8, 768]) in the checkpoint and torch.Size([60, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([8]) in the checkpoint and torch.Size([60]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


define training arguments

In [19]:
args = transformers.TrainingArguments(
    output_dir= "my_model", #directory for storing model checkpoints
    learning_rate= 2e-5,
    per_device_train_batch_size= 32,
    per_device_eval_batch_size= 32,
    num_train_epochs= 20,
    weight_decay= .01, #L2 regularization
    eval_strategy= "epoch", #model wii be evaluated at the end of each epoch
    save_strategy= "epoch",
    load_best_model_at_end= True,
)

In [20]:
trainer= transformers.Trainer(
    model= model,
    args= args,
    train_dataset= tokenized_intent["train"],
    eval_dataset = tokenized_intent["validation"],
    tokenizer= tokenizer,
    data_collator= data_collator,
    compute_metrics= compute_metrics,
)

train the model for 20 epochs

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.077852,0.777177
2,1.847600,0.703817,0.843581
3,0.603800,0.635566,0.848992
4,0.603800,0.647748,0.847024
5,0.303700,0.652064,0.851451
6,0.172300,0.693770,0.855386
7,0.106600,0.694203,0.851943
8,0.106600,0.742602,0.855878
9,0.063600,0.776879,0.854894
10,0.046400,0.817881,0.854894


Could not locate the best model at my_model/checkpoint-1080/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=7200, training_loss=0.22647883921861647, metrics={'train_runtime': 1779.1274, 'train_samples_per_second': 129.434, 'train_steps_per_second': 4.047, 'total_flos': 2267573652509760.0, 'train_loss': 0.22647883921861647, 'epoch': 20.0})

running test on test set

In [22]:
results = trainer.evaluate(tokenized_intent["test"])

In [23]:
print(results)

{'eval_loss': 0.8572596907615662, 'eval_accuracy': 0.8695359784801614, 'eval_runtime': 3.3346, 'eval_samples_per_second': 891.854, 'eval_steps_per_second': 27.889, 'epoch': 20.0}


use the model in the hub for your inputs

In [38]:
text = tokenized_intent["test"][10]["utt"]
label = tokenized_intent["test"][10]["labels"]
classifier = transformers.pipeline("sentiment-analysis" , model= "master-mahdi/intent_detection_fa")
print(text, classifier(text[:64]), sep= "\n")

config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/652M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

اولی بهم بگو ساعت توی پنج ساعت جلوتر از وقت گرینویچ چنده
[{'label': 'datetime_convert', 'score': 0.9934443235397339}]


In [27]:
id2label[38]

'datetime_convert'